In [1]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
#from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from agentic_ai.src.decision_making_agent_prompt import custom_llm_model
import torch

/Users/ritkumar17/Desktop/R&D_Project/Agentic_AI/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.set_default_device("mps")
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en")

/var/folders/00/rndss5vd6h9cdr24fwvnz6kw0000gp/T/ipykernel_68767/341949512.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en")


In [3]:
path = "/Users/ritkumar17/Desktop/R&D_Project/Agentic_AI/agentic_ai/data/chroma_db"
knowledge_base = Chroma(persist_directory=path, embedding_function=embeddings_model)

/var/folders/00/rndss5vd6h9cdr24fwvnz6kw0000gp/T/ipykernel_68767/1255448939.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  knowledge_base = Chroma(persist_directory=path, embedding_function=embeddings_model)


In [4]:
retriever = knowledge_base.as_retriever(search_kwargs={"k": 3})


In [5]:
llm = custom_llm_model()

Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.70s/it]
Some parameters are on the meta device because they were offloaded to the disk.
Device set to use mps
/Users/ritkumar17/Desktop/R&D_Project/Agentic_AI/agentic_ai/src/decision_making_agent_prompt.py:37: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  custom_llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [6]:
query = "Have we done anything on Autonomous Vehicle Navigation System"

In [ ]:
# from langchain.prompts import PromptTemplate
# from langchain.chains import create_stuff_documents_chain, create_retrieval_chain

# # Mistral-specific prompt template
# mistral_template = """[INST] 
# <<SYS>>
# You are a helpful assistant for Company XYZ. Answer strictly based on these documents:
# {context}
# <</SYS>>

# {input} 
# [/INST]"""  # Closing tag must be at end of prompt

# # Create standard PromptTemplate (not ChatPromptTemplate)
# prompt = PromptTemplate.from_template(mistral_template)

# # Create document chain
# question_answer_chain = create_stuff_documents_chain(llm, prompt)

# # Create retrieval chain
# chain = create_retrieval_chain(retriever, question_answer_chain)

# # Invoke with input
# response = chain.invoke({"input": "What is the project deadline?"})
# print(response["answer"])

In [7]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
#from langchain_openai import ChatOpenAI

#retriever = ...  # Your retriever


system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

chain.invoke({"input": query})

{'input': 'Have we done anything on Autonomous Vehicle Navigation System',
 'context': [Document(metadata={}, page_content='\n\nProject Titan: Autonomous Vehicle Navigation System\nProject Overview:\nProject Titan was focused on developing a robust autonomous driving system for self-driving cars. The primary objective was to navigate complex urban environments while ensuring the safety of passengers and pedestrians. The system needed to process data from cameras, LIDAR, and radar sensors in real time to make driving decisions.\n\nAlgorithms Tried:\nWe experimented with classical computer vision techniques like Convolutional Neural Networks (CNNs) for object detection and lane detection. Deep Reinforcement Learning (DRL) was also tested for decision-making. Ultimately, a combination of CNNs for vision and DRL for control actions provided the best results.\n\nBest Performing Algorithm:\nA hybrid model combining CNNs for object detection and Deep Reinforcement Learning (DRL) for navigatio

In [13]:
from langchain.prompts import ChatPromptTemplate

# Define a SINGLE [INST] block with system + human messages combined
system_prompt = """[INST] 
<<SYS>>
You are a helpful assistant for Company XYZ. Answer questions based on the provided documents.
<</SYS>>

### Documents:
{context}

### Question:
{input}

If the answer is in the documents, use them. Otherwise, say "The documents do not contain relevant information." [/INST]
"""

# Simplified prompt template
prompt = ChatPromptTemplate.from_messages([("system", system_prompt)])

# Format with variables
formatted_prompt = prompt.format(
    context="Project documentation text here.",
    input="What is the project deadline?"
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

response = chain.invoke({"input": query})
#print(formatted_prompt)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import create_stuff_documents_chain, create_retrieval_chain

# Define a SINGLE [INST] block with system + human messages combined
system_prompt = """[INST] 
<<SYS>>
You are a helpful assistant for Company XYZ. Answer questions based on the provided documents.
<</SYS>>

### Documents:
{context}

### Question:
{input}

If the answer is in the documents, use them. Otherwise, say "The documents do not contain relevant information." [/INST]
"""

# ✅ Use from_template() instead of from_messages()
prompt = ChatPromptTemplate.from_template(system_prompt)

# Define the LLM-based document Q&A chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Define retrieval chain using retriever and Q&A chain
chain = create_retrieval_chain(retriever, question_answer_chain)

# ✅ Define query before invoking the chain
query = "What is the project deadline?"
response = chain.invoke({"input": query})

print(response)


In [14]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
# formatted_prompt = prompt.format(context="Project documentation text here.", input="What is the project deadline?")
# print(formatted_prompt)

In [15]:
response = chain.invoke({"input": query})

In [ ]:
response

In [ ]:
response["answer"]

In [ ]:
["answer"]

In [ ]:
response_text = response["answer"].strip()
if response_text.startswith("?"):
    response_text = response_text[1:].strip()  # Remove leading "?" if present
print("🔹 Answer:", response_text)

In [ ]:
response_text

In [ ]:
system_prompt = """[INST] 
You are a helpful assistant for Company XYZ. Your task is to answer questions based on the provided project documents.

### Documents:
{context}

### Question:
{input}  
If the documents contain relevant information, answer based only on them.
If the documents do not provide an answer, say "The documents do not contain relevant information."
[/INST]"""

# Define the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),  # System prompt now uses {input}
        ("human", "{input}"),      # Human message remains the same
    ]
)

# Format with consistent variable names
formatted_prompt = prompt.format(
    context="Project documentation text here.",
    input="What is the project deadline?"
)
print(formatted_prompt)

In [ ]:
from langchain.prompts import ChatPromptTemplate

# Define a SINGLE [INST] block with system + human messages combined
system_prompt = """[INST] 
<<SYS>>
You are a helpful assistant for Company XYZ. Answer questions based on the provided documents.
<</SYS>>

### Documents:
{context}

### Question:
{input}

If the answer is in the documents, use them. Otherwise, say "The documents do not contain relevant information." [/INST]
"""

# Simplified prompt template
prompt = ChatPromptTemplate.from_messages([("system", system_prompt)])

# Format with variables
formatted_prompt = prompt.format(
    context="Project documentation text here.",
    input="What is the project deadline?"
)

print(formatted_prompt)dd